In [ ]:
%%HTML
<style type="text/css">
div.h1 {
    background-color:#e17b34; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 35px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}

div.h2 {
    background-color:#83ccd2; 
    color: white; 
    padding: 8px; 
    padding-right: 300px; 
    font-size: 35px; 
    max-width: 1500px; 
    margin: auto; 
    margin-top: 50px;
}
</style>

# <div class="h1">Motivation ☀</div>


I checked MoA because I needed it's minimum knowledge to create the model.

And I found that it seems important to carefully examine the MoA conditions(train data) ont he view of whether the MoAs score is 0 or 1.

I will summarize the snippets so that EDA can be done from this perspective.

### Our purpose of this competition

In discription of overview,

> Our task is to use the training dataset to develop an algorithm that automatically labels each case in the test set as one or more MoA classes. Note that since drugs can have multiple MoA annotations, the task is formally a multi-label classification problem.

### About MoA(mechanism of action)

Especially with omics based methods , we can identify the potential targets of the compound of interest. Using genetic perturbation in combination with the compound,we can identify genes whose knockdown or knockout abolishes the pharmacological effect of the compound.



![](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mechanism_of_action_for_beta_blockers.png/220px-Mechanism_of_action_for_beta_blockers.png)

Refered from https://en.wikipedia.org/wiki/Mechanism_of_action


Obviously, it seems important to carefully examine whether the MoAs score is 0 or 1 when the cells are given the treatment of the training set.

### Loss function

> The accuracy of solutions will be evaluated on the average value of the logarithmic loss function applied to each drug-MoA annotation pair.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html

# <div class="h2">EDA</div>

# Data overview

There are 3 data for train data. I'll check overview of them.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!ls ../input/lish-moa/

In [ ]:
train_features = pd.read_csv("../input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")

In [ ]:
print("train_features's shape:", train_features.shape)
print("train_targets_scored's shape:", train_targets_scored.shape)
print("train_targets_nonscored's shape:", train_targets_nonscored.shape)

### train_features.csv

Features for the training set. Features g- signify gene expression data, and c- signify cell viability data.

In [ ]:
train_features.head()

In [ ]:
train_features.dtypes

In [ ]:
print("Number of g- features in train_features:", len([col for col in train_features.columns if "g-" in col]))
print("Number of c- features in train_features:", len([col for col in train_features.columns if "c-" in col]))

- *cp_type* indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs

- *cp_time* and *cp_dose* indicate treatment duration (24, 48, 72 hours) and dose (high or low).

In [ ]:
sns.countplot(x="cp_type", data=train_features)

In [ ]:
sns.countplot(x="cp_time", data=train_features)

In [ ]:
sns.countplot(x="cp_dose", data=train_features)

### train_targets_scored.csv

The binary MoA targets that are scored.

In [ ]:
train_targets_scored.head()

In [ ]:
train_targets_scored.loc[0]

In [ ]:
train_targets_scored.dtypes

### train_targets_nonscored.csv: 

Additional (optional) binary MoA responses for the training data. These are not predicted nor scored.

In [ ]:
train_targets_nonscored.head()

In [ ]:
train_targets_nonscored.dtypes

## Rate of each MoAs score

I'ii check active and not active rate for each MoAs.

In [ ]:
train = train_features.merge(train_targets_scored, on='sig_id')

In [ ]:
target_cols = [col for col in train.columns if col not in train_features.columns]

In [ ]:
sig_id = []
one = []
zero = []
for col in target_cols:
    sig_id.append(col)
    one.append(len(train_features[train[col] == 1]))
    zero.append(len(train_features[train[col] == 0]))
d = {"sig_id": sig_id, "one": one, "zero": zero}
    
train_scores = pd.DataFrame.from_dict(d, orient='index').T

In [ ]:
train_scores.head()

In [ ]:
# You can extract data by index number like this.
train_scores.iloc[100]

In [ ]:
sns.scatterplot(data=train_scores, x="one", y="zero")

Number of record which MoAs score is 1 seems be less than approximately 1% of number of records MoAs score is 0. 
 
And the rate wheather 0 or 1 seems be constant.

In [ ]:
train_scores[train_scores["one"] == 0]

## Gene data distribution for each MoAs

I'll check gene data distribution by displot. Distribution of records which MoAs score is 0 is seems similar inspite of value of g-"*".

In [ ]:
sig_id = "5-alpha_reductase_inhibitor"
gene_data = "g-0"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][gene_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][gene_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

In [ ]:
sig_id = "5-alpha_reductase_inhibitor"
gene_data = "g-5"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][gene_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][gene_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

Distribution of records which MoAs score is 0 is also seems similar inspite of value of "sig_id", too.

In [ ]:
sig_id = "vegfr_inhibitor"
gene_data = "g-0"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][gene_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][gene_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

In [ ]:
sig_id = "gonadotropin_receptor_agonist"
gene_data = "g-0"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][gene_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][gene_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

## Cell data distribution for each MoAs

I'll check cell data distribution by displot. Distribution of records which MoAs score is 0 is seems similar inspite of value of c-"*" and "sig_id".

In [ ]:
sig_id = "5-alpha_reductase_inhibitor"
cell_data = "c-0"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][cell_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][cell_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

In [ ]:
sig_id = "5-alpha_reductase_inhibitor"
cell_data = "c-5"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][cell_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][cell_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

In [ ]:
sig_id = "vegfr_inhibitor"
cell_data = "c-50"

print(train_scores[train_scores["sig_id"] == sig_id])

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)

g1 = sns.distplot(train_features[train[sig_id] == 1][cell_data], ax=ax1)
g2 = sns.distplot(train_features[train[sig_id] == 0][cell_data], ax=ax2)

g1.set_title("one")
g2.set_title("zero")

-----------
### If you like this kernel, please vote! Thank you! 😹